<a href="https://colab.research.google.com/github/PyxAI/Data-Science-Notebooks/blob/master/The_OneNet_sharing_lower_levels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experminet, I took two resnet18 pretrained networks and made them share the extracted features between them at lower layers of the network, rather than just at the classifier level.

I've tried both Linear and Convolutional layers to be the ones shared between the networks.

All of the experiments had a negative effect on the loss and accuracy.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

import torch
import torch.nn.functional as F
from torch import nn
from torchvision import models
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from google.colab import drive

if torch.cuda.is_available():
  device = torch.device('cuda')
  dtype = torch.cuda.FloatTensor
else:
  device = torch.device('cpu')
  dtype = torch.FloatTensor


def validate(network, dset):
  network.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    loss = 0
    for x, y in dset:
      x = x.to(device)
      y = y.to(device)
      output = network(x)
      loss += loss_fn(output, y)
      _, predicted = torch.max(output.data, 1)
      total += y.size(0)
      correct += (predicted == y).sum().item()
    loss/= len(dset)
    print ("validation loss: {}".format(loss))
    print ("accuracy here is: {}".format(100 * correct / total))
  return loss.detach().item()

drive.mount('/content/gdrive')
!mkdir /content/weights
!cp /content/gdrive/My\ Drive/weights/resnet18_cifar.pt /content/weights/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class Squeezer(nn.Module):
  def __init__(self):
    super(Squeezer,self).__init__()
  def forward(self, x):
      return x.squeeze()

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

def run_train(network):
  loss_arr = []
  try:
    for epoch in range(epochs):
      for iteration, (x, y) in enumerate(train):
        network.train()
        x = x.to(device)
        y = y.to(device)
        optim.zero_grad()
        output = network(x)
        loss = loss_fn(output, y)
        loss.backward()
        if (iteration % print_every == 0):
          print ("epoch: {}, iter:{}".format(epoch, iteration))
          loss_arr.append(validate(network, val))
        optim.step()
  except KeyboardInterrupt:
    plt.plot(loss_arr)
    plt.show()



##Settings

In [0]:
#Training
batch_size = 64
lr = 2e-5
epochs=200
print_every = 100

#Dataset
cifar_root = '/content/data/cifar10'
cifar_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
    ])

#Dataset

In [0]:
#Take Cifar10
cifar = datasets.CIFAR10(cifar_root, train=True, download=True, transform=cifar_transform)

#split to train - val
train_set, val_set = torch.utils.data.random_split(cifar, [int(len(cifar)*0.9), int(len(cifar)*0.1)])
train = DataLoader(train_set, batch_size = batch_size, shuffle=True, num_workers = 4, drop_last =True)
val = DataLoader(val_set, batch_size = batch_size, shuffle=True, num_workers = 4, drop_last =True)

#test set
cifar = datasets.CIFAR10(cifar_root, train=False, download=True, transform=cifar_transform)
test = DataLoader(cifar, batch_size = batch_size, shuffle=True, num_workers = 4, drop_last =True)

Extracting /content/data/cifar10/cifar-10-python.tar.gz to /content/data/cifar10
Files already downloaded and verified


##Model setup

In [0]:
%%capture

#Loading resnet with pretrained weights from CIFAR10
resnet18A = models.resnet18(pretrained=False)
resnet18A._modules['fc']= nn.Linear(512, 10) #To make it CIFARable
resnet18A.load_state_dict(torch.load('/content/gdrive/My Drive/weights/ready_resnet18_cifar.pt', map_location=torch.device(device)))
resnet18A.fc = Identity()

#Resnet with ImageNet
resnet18B = models.resnet18(pretrained=True)
resnet18B.fc = Identity()

resnet18A.to(device)
resnet18B.to(device)

Shutting down grad on everything but FC layer

In [0]:
#Shutting down grad on everything but FC layer
for param in resnet18A.named_parameters():
  if not any(['body' in param[0]]):
    param[1].requires_grad=False

#Shutting down grad on everything but FC layer
for param in resnet18B.named_parameters():
  if not any(['body' in param[0]]):
    param[1].requires_grad=False


In [0]:
# Separation the different steps 

t0 = nn.Sequential(
    resnet18A.conv1,
    resnet18A.bn1,
    resnet18A.relu,
    resnet18A.maxpool
)

t1 = resnet18A.layer1
t2 = resnet18A.layer2
t3 = resnet18A.layer3
t4 = resnet18A.layer4
head = nn.Sequential(
    resnet18A.avgpool,
)

r0 = nn.Sequential(
    resnet18B.conv1,
    resnet18B.bn1,
    resnet18B.relu,
    resnet18B.maxpool
)

r1 = resnet18B.layer1
r2 = resnet18B.layer2
r3 = resnet18B.layer3
r4 = resnet18B.layer4
rhead = nn.Sequential(
    resnet18B.avgpool,
)

In [0]:
class FriendNetConv(nn.Module):
  def __init__(self):
    super(FriendNetConv, self).__init__()

    self.conv1 = nn.Sequential(
        nn.LeakyReLU(),
        nn.Conv2d(512,256, kernel_size=(1,1)),
    )

    self.body = nn.Sequential(
            nn.LeakyReLU(),
            Squeezer(),
            nn.Linear(512*2, 10),
        )
  # We run forward on each layer of each network seperatley.
  def forward(self, x):
    x11 = t0(x)
    x12 = r0(x)
    x21 = t1(x11)
    x22 = r1(x12)
    x31 = t2(x21)
    x32 = r2(x22)
    x41 = t3(x31)
    x42 = r3(x32)
    # Concatenating and running conv layer
    x1 = torch.cat((x41, x42),dim=1)
    x1 = self.conv1(x1)
    # Feeding back to the original structure
    x51 = t4(x1)
    x52 = r4(x1)
    x61 = head(x51)
    x62 = rhead(x52)
    x6 = torch.cat((x61, x62), dim=1)
    x = self.body(x6)
    return x




In [0]:
class FriendNetLinear(nn.Module):
  def __init__(self):
    super(FriendNetLinear, self).__init__()
    
    self.lin1 = nn.Sequential(
        nn.LeakyReLU(),
        nn.Linear(2048, 256, bias=True),
        #nn.LeakyReLU(),
    )
    
    self.body = nn.Sequential(
            nn.LeakyReLU(),
            Squeezer(),
            nn.Linear(512*2, 10),
        )
    self.bn1 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

  def forward(self, x):
    x11 = t0(x)
    x12 = r0(x)
    x21 = t1(x11)
    x22 = r1(x12)
    x31 = t2(x21)
    x32 = r2(x22)
    x41 = t3(x31)
    x42 = r3(x32)
    x1 = torch.cat((x41, x42),dim=1)
    x1 = x1.view(batch_size,-1,1,1).squeeze() # to shape: (batch_size, 2048)
    x1 = self.lin1(x1)
    x1 = x1.view(batch_size,-1,1,1) # to shape: (batch_size, 256,1,1)
    x1 = self.bn1(x1)
    x51 = t4(x1)
    x52 = r4(x1)
    x61 = head(x51)
    x62 = rhead(x52)
    x6 = torch.cat((x61, x62), dim=1)
    x = self.body(x6)
    return x




In [0]:
friend = FriendNetLinear()

optim = torch.optim.Adam(friend.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

friend.to(device)

run_train(friend)

In [0]:
friend = FriendNetConv()

optim = torch.optim.Adam(friend.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

friend.to(device)
run_train(friend)